In [1]:
!python -V

Python 3.11.7


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

In [6]:

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='/home/yoda/Desktop/Placement/github/Mlops-Zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1716829779361, experiment_id='3', last_update_time=1716829779361, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [9]:
len(df_train), len(df_val)

(3009173, 2855951)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

5.247604171320468

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer","cristian")

    mlflow.log_param("train-data-path","./data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path","./data/yellow_tripdata_2023-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha",alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse",rmse)

In [13]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/yoda/anaconda3/envs/zoomcamp/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [23:40:23] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.44001                           
[1]	validation-rmse:7.28370                           
[2]	validation-rmse:6.48059                           
[3]	validation-rmse:5.93688                           
[4]	validation-rmse:5.57563                           
[5]	validation-rmse:5.33796                           
[6]	validation-rmse:5.18004                           
[7]	validation-rmse:5.07566                           
[8]	validation-rmse:5.00470                           
[9]	validation-rmse:4.95559                           
[10]	validation-rmse:4.92126                          
[11]	validation-rmse:4.89632                          
[12]	validation-rmse:4.87689                          
[13]	validation-rmse:4.86265                          
[14]	validation-rmse:4.85112                          
[15]	validation-rmse:4.84193                          
[16]	validation-rmse:4.83437                          
[17]	validation-rmse:4.82754                          
[18]	valid

/home/yoda/anaconda3/envs/zoomcamp/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [00:05:41] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.65512                                                         
[1]	validation-rmse:5.48697                                                         
[2]	validation-rmse:5.12752                                                         
[3]	validation-rmse:5.00840                                                         
[4]	validation-rmse:4.96157                                                         
[5]	validation-rmse:4.93457                                                         
[6]	validation-rmse:4.92173                                                         
[7]	validation-rmse:4.91199                                                         
[8]	validation-rmse:4.90765                                                         
[9]	validation-rmse:4.89954                                                         
[10]	validation-rmse:4.89566                                                        
[11]	validation-rmse:4.89217                                     

...

[0]	validation-rmse:5.44720                                                        
[1]	validation-rmse:4.88660                                                        
[2]	validation-rmse:4.79584                                                        
[3]	validation-rmse:4.76737                                                        
[4]	validation-rmse:4.75338                                                        
[5]	validation-rmse:4.74781                                                        
[6]	validation-rmse:4.74225                                                        
[7]	validation-rmse:4.73732                                                        
[8]	validation-rmse:4.73316                                                        
[9]	validation-rmse:4.72887                                                        
[10]	validation-rmse:4.72011                                                       
[11]	validation-rmse:4.71607                                                

/home/yoda/anaconda3/envs/zoomcamp/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [00:42:06] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.65394                                                         
[1]	validation-rmse:7.58870                                                         
[2]	validation-rmse:6.79966                                                         
[3]	validation-rmse:6.22525                                                         
[4]	validation-rmse:5.81403                                                         
[5]	validation-rmse:5.52257                                                         
[6]	validation-rmse:5.31788                                                         
[7]	validation-rmse:5.17315                                                         
[8]	validation-rmse:5.07088                                                         
[9]	validation-rmse:4.99904                                                         
[10]	validation-rmse:4.94626                                                        
[11]	validation-rmse:4.90674                                     

/home/yoda/anaconda3/envs/zoomcamp/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:14:03] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



: 